# 如何尝试修复输出解析中的错误

:::info 前提条件

本指南假定您熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)
- [提示模板](/docs/concepts/prompt_templates)
- [将可运行对象串联在一起](/docs/how_to/sequence/)

:::

大型语言模型（LLMs）并非完美，有时会生成不符合预期格式的输出。为了处理此类错误，我们可以使用 [`OutputFixingParser`](https://api.js.langchain.com/classes/langchain.output_parsers.OutputFixingParser.html)。该输出解析器封装了另一个输出解析器，当第一个解析器失败时，它会调用另一个LLM来尝试修复错误。

具体来说，我们可以将格式错误的输出与格式化说明一起传递给模型，并要求其进行修正。

在本示例中，我们将使用 [`StructuredOutputParser`](https://api.js.langchain.com/classes/langchain_core.output_parsers.StructuredOutputParser.html)，它可以根据Zod模式验证输出。如果我们传递一个不符合模式的结果，会发生以下情况：

In [1]:
import { z } from "zod";
import { RunnableSequence } from "@langchain/core/runnables";
import { StructuredOutputParser } from "@langchain/core/output_parsers";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const zodSchema = z.object({
  name: z.string().describe("name of an actor"),
  film_names: z.array(z.string()).describe("list of names of films they starred in"),
});

const parser = StructuredOutputParser.fromZodSchema(zodSchema);

const misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}";

await parser.parse(misformatted);

Error: Failed to parse. Text: "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}". Error: SyntaxError: Expected property name or '}' in JSON at position 1 (line 1 column 2)

现在我们可以构建并使用一个 `OutputFixingParser`。这个输出解析器接受另一个输出解析器作为参数，同时还需要一个大语言模型（LLM），用于尝试纠正任何格式上的错误。

In [3]:
import { ChatAnthropic } from "@langchain/anthropic";

import { OutputFixingParser } from "langchain/output_parsers";

const model = new ChatAnthropic({
  model: "claude-3-sonnet-20240229",
  maxTokens: 512,
  temperature: 0.1,
});

const parserWithFix = OutputFixingParser.fromLLM(model, parser);

await parserWithFix.parse(misformatted);

{
  name: "Tom Hanks",
  film_names: [
    "Forrest Gump",
    "Saving Private Ryan",
    "Cast Away",
    "Catch Me If You Can"
  ]
}

有关不同参数和选项的更多信息，请查看我们的[API 参考文档](https://api.js.langchain.com/classes/langchain.output_parsers.OutputFixingParser.html)。